# Implementing Word2Vec Model using Skip-Gram 

## Importing the Necesssary Stuff

In [1]:
import numpy as np 
import string 
from nltk.corpus import stopwords
import re

<img src="https://media.geeksforgeeks.org/wp-content/uploads/Skip-gram-architecture-2.jpg">

Let's define some variables :

V    Number of unique words in our corpus of text ( Vocabulary )<br>
x    Input layer (One hot encoding of our input word ). <br>
N    Number of neurons in the hidden layer of neural network<br>
W    Weights between input layer and hidden layer<br>
W'   Weights between hidden layer and output layer<br>
y    A softmax output layer having probabilities of every word in our vocabulary

## Softmax Function

In [2]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum() 
   

## Word2Vec Class

### Contains Functions for Forward Propogation, Backward Propogation, Training and Predicting the words.

In <u><b>Forward Propogation</b></u>, We multiply one hot encoding of centre word (denoted by x) with the first weight matrix W to get hidden layer matrix h (of size N x 1). We then multiply the hidden layer vector h with second weight matrix W’ to get a new matrix u. We Then obtain our loss function, Which comes out to be <br><img src="https://miro.medium.com/max/994/1*XPhzBnf1xEb0u67qazx9nA.png"><br>
E being our Loss Function.<br>
In <u><b>Backward Propogation</b></u>, We find the partial derivatives of our loss function with respect to W and W’ to apply gradient descent algorithm.


In [3]:
class word2vec(object): 
    def __init__(self): 
        self.N = 10
        self.X_train = [] 
        self.y_train = [] 
        self.window_size = 2
        self.alpha = 0.001
        self.words = [] 
        self.word_index = {} 
   
    def initialize(self,V,data): 
        self.V = V 
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 
           
        self.words = data 
        for i in range(len(data)): 
            self.word_index[data[i]] = i 
   
       
    def feed_forward(self,X): 
        self.h = np.dot(self.W.T,X).reshape(self.N,1) 
        self.u = np.dot(self.W1.T,self.h) 
        #print(self.u) 
        self.y = softmax(self.u)   
        return self.y 
           
    def backpropagate(self,x,t): 
        e = self.y - np.asarray(t).reshape(self.V,1) 
        # e.shape is V x 1 
        dLdW1 = np.dot(self.h,e.T) 
        X = np.array(x).reshape(self.V,1) 
        dLdW = np.dot(X, np.dot(self.W1,e).T) 
        self.W1 = self.W1 - self.alpha*dLdW1 
        self.W = self.W - self.alpha*dLdW 
           
    def train(self,epochs): 
        for x in range(1,epochs):         
            self.loss = 0
            for j in range(len(self.X_train)): 
                self.feed_forward(self.X_train[j]) 
                self.backpropagate(self.X_train[j],self.y_train[j]) 
                C = 0
                for m in range(self.V): 
                    if(self.y_train[j][m]): 
                        self.loss += -1*self.u[m][0] 
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u))) 
            print("epoch ",x, " loss = ",self.loss) 
            self.alpha *= 1/( (1+self.alpha*x) ) 
              
    def predict(self,word,number_of_predictions): 
        if word in self.words: 
            index = self.word_index[word] 
            X = [0 for i in range(self.V)] 
            X[index] = 1
            prediction = self.feed_forward(X) 
            output = {} 
            for i in range(self.V): 
                output[prediction[i][0]] = i 
               
            top_context_words = [] 
            for k in sorted(output,reverse=True): 
                top_context_words.append(self.words[output[k]]) 
                if(len(top_context_words)>=number_of_predictions): 
                    break
       
            return top_context_words 
        else: 
            print("Word not found in dicitonary") 

## Functions for Preparing and Preprocessing Data

In [4]:
def preprocessing(corpus): 
    stop_words = set(stopwords.words('english'))     
    training_data = [] 
    sentences = corpus.split(".") 
    for i in range(len(sentences)): 
        sentences[i] = sentences[i].strip() 
        sentence = sentences[i].split() 
        x = [word.strip(string.punctuation) for word in sentence 
                                     if word not in stop_words] 
        x = [word.lower() for word in x] 
        training_data.append(x) 
    return training_data 
       
   
def prepare_data_for_training(sentences,w2v): 
    data = {} 
    for sentence in sentences: 
        for word in sentence: 
            if word not in data: 
                data[word] = 1
            else: 
                data[word] += 1
    V = len(data) 
    data = sorted(list(data.keys())) 
    vocab = {} 
    for i in range(len(data)): 
        vocab[data[i]] = i 
       
    #for i in range(len(words)): 
    for sentence in sentences: 
        for i in range(len(sentence)): 
            center_word = [0 for x in range(V)] 
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)] 
              
            for j in range(i-w2v.window_size,i+w2v.window_size): 
                if i!=j and j>=0 and j<len(sentence): 
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word) 
            w2v.y_train.append(context) 
    w2v.initialize(V,data) 
   
    return w2v.X_train,w2v.y_train 

## Source Code

In [5]:
corpus = "" 
corpus += "anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution"
epochs = 1000
  
training_data = preprocessing(corpus) 
w2v = word2vec() 
  
prepare_data_for_training(training_data,w2v) 
w2v.train(epochs)  
  
print(w2v.predict("term",3))     

1.29210593184752
epoch  484  loss =  141.29056401378705
epoch  485  loss =  141.28902844762388
epoch  486  loss =  141.2874991942975
epoch  487  loss =  141.2859762150668
epoch  488  loss =  141.2844594715055
epoch  489  loss =  141.28294892550002
epoch  490  loss =  141.28144453924565
epoch  491  loss =  141.2799462752441
epoch  492  loss =  141.2784540962994
epoch  493  loss =  141.2769679655161
epoch  494  loss =  141.2754878462951
epoch  495  loss =  141.2740137023315
epoch  496  loss =  141.2725454976114
epoch  497  loss =  141.2710831964087
epoch  498  loss =  141.2696267632827
epoch  499  loss =  141.26817616307508
epoch  500  loss =  141.26673136090713
epoch  501  loss =  141.26529232217675
epoch  502  loss =  141.26385901255594
epoch  503  loss =  141.26243139798834
epoch  504  loss =  141.2610094446858
epoch  505  loss =  141.25959311912683
epoch  506  loss =  141.25818238805297
epoch  507  loss =  141.2567772184668
epoch  508  loss =  141.255377577629
epoch  509  loss =  141

The Model Predicts 'anarchism', 'originated' and 'class' to have similar meaning to that of 'term' which is somewhat true.<br>
Hence we can say that this model is Functional.